# PRMT-2460 OCTOBER - Produce transfer level report of failures for November

## Context
Produce a transfer level report for failures using a weeks worth of data
- Date range: November 18th - November 24th (7 days)
- No cutoff - but set overflow as late as possible (gives transfers earlier on more time to receive messages)

In [37]:
import pandas as pd 
import numpy as np
import paths, data
from sklearn.preprocessing import MultiLabelBinarizer
from datetime import datetime
from data.practice_metadata import read_asid_metadata

In [38]:
error_code_lookup_file = pd.read_csv(data.gp2gp_response_codes.path)
error_code_lookup = error_code_lookup_file.set_index("ErrorCode")["ErrorName"]

In [93]:
asid_lookup = read_asid_metadata("prm-gp2gp-ods-metadata-preprod", "v2/2021/11/organisationMetadata.json")
transfer_file_location = "s3://prm-gp2gp-transfer-data-preprod/v6/2021/10/2021-10-transfers.parquet"

transfers_raw = pd.read_parquet(transfer_file_location)
transfers = transfers_raw\
    .join(asid_lookup.add_prefix("requesting_"), on="requesting_practice_asid", how="left")\
    .join(asid_lookup.add_prefix("sending_"), on="sending_practice_asid", how="left")\

transfers["status"] = transfers["status"].str.replace("_", " ").str.capitalize()

len(transfers)

259504

In [114]:
# Ensure we only look at transfers between 18th and 24th November
first_half_date_filter_bool_start = transfers["date_requested"] >= datetime(2021, 11, 18)
first_half_date_filter_bool_end = transfers["date_requested"] < datetime(2021, 11, 25)
transfers_within_date_range = transfers
len(transfers_within_date_range)

259504

In [115]:
# Look for transfers that have status Technical Failure or Unclassified Failure
failed_transfers_bool = (transfers_within_date_range["status"] == "Technical failure") | (transfers_within_date_range["status"] == "Unclassified failure")
failed_transfers = transfers_within_date_range[failed_transfers_bool]
len(failed_transfers)

16360

In [116]:
# Remove Nan and duplicate occurrences in error codes
def filter_error_codes(error_codes_column):
    return error_codes_column.apply(lambda sender_error_codes: [sender_error_code for sender_error_code in set(sender_error_codes) if not np.isnan(sender_error_code)])

# This is important otherwise notebook will complain about mutating original list/dataset in fuction calls below
failed_transfers = failed_transfers.copy()

failed_transfers["unique_sender_error_codes"] = filter_error_codes(failed_transfers["sender_error_codes"])
failed_transfers["unique_final_error_codes"] = filter_error_codes(failed_transfers["final_error_codes"])
failed_transfers["unique_intermediate_error_codes"] = filter_error_codes(failed_transfers["intermediate_error_codes"])
failed_transfers.head(1)

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,...,requesting_practice_name,requesting_ccg_ods_code,requesting_ccg_name,sending_practice_ods_code,sending_practice_name,sending_ccg_ods_code,sending_ccg_name,unique_sender_error_codes,unique_final_error_codes,unique_intermediate_error_codes
1,AF8BCDC6-345A-4BD4-9D46-C2F51C2ADAFC,NaN,200000009178,200000006243,EMIS,EMIS,"[nan, nan, nan]",[],[29],Technical failure,...,THE HERON MEDICAL PRACTICE,91Q,NHS KENT AND MEDWAY CCG,G82704,THE CHURCH VIEW PRACTICE,91Q,NHS KENT AND MEDWAY CCG,[],[],[29]


In [117]:
import paths, data
error_code_lookup_file = pd.read_csv(data.gp2gp_response_codes.path)
error_code_lookup = error_code_lookup_file.set_index("ErrorCode")["ErrorName"]

In [118]:
# Add descriptions from lookup to error code columns
def convert_error_list_to_formatted_string_list_with_error_lookup(error_code_list):
    return ", ".join([str(error_code.astype(int))+" - "+str(error_code_lookup[error_code]) for error_code in set(error_code_list) if not np.isnan(error_code)])

failed_transfers["unique_sender_error_codes_with_description"] = failed_transfers["unique_sender_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
failed_transfers["unique_final_error_codes_with_description"] = failed_transfers["unique_final_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
failed_transfers["unique_intermediate_error_codes_with_description"] = failed_transfers["unique_intermediate_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)

In [119]:
# Prune columns we want to keep for the report
columns_to_keep = [
    'sending_practice_ods_code',
    'sending_supplier',
    'requesting_practice_ods_code',    
    'requesting_supplier',
    'conversation_id', 
    'date_requested', 
    'status',
    'failure_reason', 
    'unique_sender_error_codes_with_description', 
    'unique_final_error_codes_with_description',
    'unique_intermediate_error_codes_with_description']

failed_transfers_with_filtered_and_renamed_columns = failed_transfers[columns_to_keep]

failed_transfers_with_filtered_and_renamed_columns=failed_transfers_with_filtered_and_renamed_columns.rename({
    "sending_practice_ods_code":"Sending Practice ODS",
    "sending_supplier":"Sending Supplier",
    "requesting_practice_ods_code":"Requesting Practice ODS",
    "requesting_supplier":"Requesting Supplier",
    "conversation_id":"Conversation ID",
    "date_requested":"Date Requested",
    "status":"Status",
    "failure_reason":"Failure Reason",
    "unique_sender_error_codes_with_description":"Unique Sender Errors",
    "unique_final_error_codes_with_description":"Unique Final Errors",
    "unique_intermediate_error_codes_with_description":"Unique Intermediate Errors"
    },axis=1)

failed_transfers_with_filtered_and_renamed_columns.head(2)

,Sending Practice ODS,Sending Supplier,Requesting Practice ODS,Requesting Supplier,Conversation ID,Date Requested,Status,Failure Reason,Unique Sender Errors,Unique Final Errors,Unique Intermediate Errors
1,G82704,EMIS,G82090,EMIS,AF8BCDC6-345A-4BD4-9D46-C2F51C2ADAFC,2021-10-08 12:58:48.261,Technical failure,COPC(s) not acknowledged,,,29 - LM reassembly
9,E85122,SystmOne,H85069,EMIS,6D752551-2189-4019-9493-24ED4BE61A9F,2021-10-12 12:17:20.999,Technical failure,Final error,20 - Spine error,25 - Timeout,


In [120]:
data = {
    'unique_sender_error_codes_count': failed_transfers.explode("unique_sender_error_codes").value_counts("unique_sender_error_codes"),
    'unique_final_error_codes_count': failed_transfers.explode("unique_final_error_codes").value_counts("unique_final_error_codes"),
    'unique_intermediate_error_codes_count': failed_transfers.explode("unique_intermediate_error_codes").value_counts("unique_intermediate_error_codes")
    }
df = pd.DataFrame(data=data)

print("November 17-25 after fix [total transfers:"+str(len(transfers_within_date_range))+", total failed transfers: "+str(len(failed_transfers))+"]")
df.sort_values(by=["unique_sender_error_codes_count", "unique_final_error_codes_count", "unique_intermediate_error_codes_count"], ascending=False)

November 17-25 after fix [total transfers:259504, total failed transfers: 16360]


,unique_sender_error_codes_count,unique_final_error_codes_count,unique_intermediate_error_codes_count
20.0,4365.0,NaN,NaN
30.0,1743.0,110.0,2.0
19.0,990.0,NaN,NaN
14.0,491.0,NaN,NaN
23.0,491.0,NaN,NaN
10.0,488.0,NaN,NaN
6.0,209.0,NaN,NaN
7.0,55.0,NaN,NaN
99.0,24.0,414.0,NaN
31.0,NaN,1927.0,NaN


In [121]:
df.sum(axis=0)

unique_sender_error_codes_count          8856.0
unique_final_error_codes_count           5597.0
unique_intermediate_error_codes_count    1719.0
dtype: float64

In [122]:
# Combine all errors into single cell to use for calculations in high level report method below
def convert_error_list_to_tuple_with_error_lookup_and_type(error_code_list, error_code_type):
    return [(error_code_type, error_code, error_code_lookup[error_code]) for error_code in set(error_code_list) if not np.isnan(error_code)]

def combine_error_codes_into_single_tuple(row):
    sender_list = convert_error_list_to_tuple_with_error_lookup_and_type(row["sender_error_codes"], "Sender")
    intermediate_list = convert_error_list_to_tuple_with_error_lookup_and_type(row["intermediate_error_codes"], "COPC")
    final_list = convert_error_list_to_tuple_with_error_lookup_and_type(row["final_error_codes"], "Final")
    full_error_code_list = sender_list + intermediate_list + final_list
    if len(full_error_code_list) == 0:
        return tuple([("No Error Code", "No Error", "N/A")])
    else:
        return tuple(full_error_code_list)

transfers_within_date_range_with_combined_errors = transfers_within_date_range.copy()

transfers_within_date_range_with_combined_errors["all_error_codes"] = transfers_within_date_range_with_combined_errors.apply(combine_error_codes_into_single_tuple, axis=1)
transfers_within_date_range_with_combined_errors.head(1)

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,...,last_sender_message_timestamp,requesting_practice_ods_code,requesting_practice_name,requesting_ccg_ods_code,requesting_ccg_name,sending_practice_ods_code,sending_practice_name,sending_ccg_ods_code,sending_ccg_name,all_error_codes
0,949AFE9C-889B-481A-B19B-25F5C76D1C67,NaN,669024222046,200000000124,EMIS,EMIS,[nan],[],[],Process failure,...,2021-10-17 23:14:58.539,P81002,LANCASTER MEDICAL PRACTICE,01K,NHS MORECAMBE BAY CCG,K84051,WHITE HORSE MEDICAL PRACTICE,10Q,NHS OXFORDSHIRE CCG,"((No Error Code, No Error, N/A),)"


In [123]:
# Format the errors to be readable strings
transfers_within_date_range_with_combined_errors["unique_sender_error_codes"] = filter_error_codes(transfers_within_date_range_with_combined_errors["sender_error_codes"])
transfers_within_date_range_with_combined_errors["unique_final_error_codes"] = filter_error_codes(transfers_within_date_range_with_combined_errors["final_error_codes"])
transfers_within_date_range_with_combined_errors["unique_intermediate_error_codes"] = filter_error_codes(transfers_within_date_range_with_combined_errors["intermediate_error_codes"])

transfers_within_date_range_with_combined_errors["unique_sender_error_codes_with_description"] = transfers_within_date_range_with_combined_errors["unique_sender_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
transfers_within_date_range_with_combined_errors["unique_final_error_codes_with_description"] = transfers_within_date_range_with_combined_errors["unique_final_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
transfers_within_date_range_with_combined_errors["unique_intermediate_error_codes_with_description"] = transfers_within_date_range_with_combined_errors["unique_intermediate_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
transfers_within_date_range_with_combined_errors.head(1)

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,...,sending_practice_name,sending_ccg_ods_code,sending_ccg_name,all_error_codes,unique_sender_error_codes,unique_final_error_codes,unique_intermediate_error_codes,unique_sender_error_codes_with_description,unique_final_error_codes_with_description,unique_intermediate_error_codes_with_description
0,949AFE9C-889B-481A-B19B-25F5C76D1C67,NaN,669024222046,200000000124,EMIS,EMIS,[nan],[],[],Process failure,...,WHITE HORSE MEDICAL PRACTICE,10Q,NHS OXFORDSHIRE CCG,"((No Error Code, No Error, N/A),)",[],[],[],,,


In [124]:
def generate_high_level_table(transfers_sample):

    # Create High level table
    high_level_table=transfers_sample.fillna("N/A").groupby(["requesting_supplier","sending_supplier","status","failure_reason","all_error_codes", "unique_final_error_codes_with_description", "unique_sender_error_codes_with_description", "unique_intermediate_error_codes_with_description"]).agg({"conversation_id":"count"})
    high_level_table=high_level_table.rename({"conversation_id":"number of transfers"},axis=1).reset_index()

    # Count % of transfers
    total_number_transfers = transfers_sample.shape[0]
    high_level_table["% of transfers"]=(high_level_table["number of transfers"]/total_number_transfers).multiply(100)
    
    # Count % of technical failures
    technical_failed_transfers_bool = transfers_sample["status"] == "Technical failure"
    technical_failed_transfers = transfers_sample[technical_failed_transfers_bool]
    total_number_technical_failed_transfers = technical_failed_transfers.shape[0]
    technical_failed_transfers_table_bool = high_level_table["status"] == "Technical failure"
    high_level_table.loc[technical_failed_transfers_table_bool, "% of technical failures"]=(high_level_table.loc[technical_failed_transfers_table_bool, "number of transfers"]/total_number_technical_failed_transfers).multiply(100)
    
    # Count by supplier pathway
    supplier_pathway_counts = transfers_sample.fillna("Unknown").groupby(by=["sending_supplier", "requesting_supplier"]).agg({"conversation_id": "count"})["conversation_id"]
    high_level_table["% supplier pathway"]=high_level_table.apply(lambda row: row["number of transfers"]/supplier_pathway_counts.loc[(row["sending_supplier"],row["requesting_supplier"])],axis=1).multiply(100)

    # % of error codes column
    no_error_tuple = tuple([("No Error Code", "No Error", "N/A")])
    error_code_bool = transfers_sample["all_error_codes"]!=no_error_tuple
    total_number_of_error_code_combinations=error_code_bool.sum()
    table_error_code_bool = high_level_table["all_error_codes"]!=no_error_tuple
    high_level_table.loc[table_error_code_bool,"% of all errors"]=(high_level_table.loc[table_error_code_bool, "number of transfers"]/total_number_of_error_code_combinations).multiply(100)
    
    # Select and re-order table
    grouping_columns_order=["requesting_supplier","sending_supplier","status","failure_reason", "unique_final_error_codes_with_description", "unique_sender_error_codes_with_description", "unique_intermediate_error_codes_with_description"]
    counting_columns_order=["number of transfers","% of transfers","% of technical failures","% supplier pathway"]
    high_level_table=high_level_table[grouping_columns_order+counting_columns_order].sort_values(by="number of transfers",ascending=False)
    
    # Rename the columns
    high_level_table=high_level_table.rename({
        "sending_supplier":"sending supplier",
        "requesting_supplier":"requesting supplier",
        "status":"status",
        "failure_reason":"failure reason",
        "unique_final_error_codes_with_description":"unique final errors",
        "unique_sender_error_codes_with_description":"unique sender errors",
        "unique_intermediate_error_codes_with_description":"unique intermediate errors"
        },axis=1)
    
    # Replace all N/A with empty strings primarily for error columns
    high_level_table=high_level_table.replace("N/A", "")
    
    return high_level_table

In [127]:
with pd.ExcelWriter("PRMT-2460-Error-code-combination-table-October.xlsx") as writer:
     generate_high_level_table(transfers_within_date_range_with_combined_errors).to_excel(writer, sheet_name="Error Code Combination Table",index=False)

In [90]:
with pd.ExcelWriter("PRMT-2460-Transfer-level-report-of-failures-for-November-18-to-November-24-2021.xlsx") as writer:
    failed_transfers_with_filtered_and_renamed_columns.to_excel(writer, sheet_name="Transfer level failures",index=False)